In [9]:
import numpy as np

import reactord as rd

## Mixture objects

To generate a mixture, first it is necessary to create the Substance objects that will be in the mixture. For instance:

In [10]:
so2 = rd.Substance.from_thermo_database("SO2")
o2 = rd.Substance.from_thermo_database("O2")
so3 = rd.Substance.from_thermo_database("SO3")
co2 = rd.Substance.from_thermo_database("CO2")

Now we create a mixture, introducing the objects as a dictionary. In this case, an instance of the IdealGas class is created:

In [11]:
gas_mixture = rd.mix.IdealGas(A=so2, B=o2, C=so3, Inert=co2)

# The mixture is a list of substances to which we can access:
print(gas_mixture.substances[0].normal_boiling_point)
print(gas_mixture.substances[1].name)
print(gas_mixture.substances[2].critical_pressure)

263.1
oxygen
8207325.0


Given the number of moles of each substance in the mixture, molar fractions can be calculated. Additionally, to calculate concentrations (**mol/m³**), temperature (**K**) and pressure (**Pa**) are required:

In [12]:
moles = [1.5, 2, 0.5, 3]

molar_fractions = np.round(gas_mixture.mol_fractions(moles), 4)
print("molar fractions: ", molar_fractions, sum(molar_fractions) == 1)

temperature = 298.15  # K
pressure = 101325  # Pa
concentrations = np.round(
    gas_mixture.concentrations(moles, temperature, pressure), 2
)
print("Concentrations (mol/m³): ", concentrations)

molar fractions:  [0.2143 0.2857 0.0714 0.4286] True
Concentrations (mol/m³):  [ 8.76 11.68  2.92 17.52]


The IdealGas class also has methods to:

-  Calculate the molar volume (mol/m³)
-  Calculate the heat capacity (J/K)
-  Establish the standard formation enthalpies of the substances in the mixture (J/(mol.K)
-  Calculate a term to correct the formation enthalpies at non-standard conditions (J/(mol.K)

In [13]:
moles = [0.5, 0.2, 0.1, 0.2]
temperature = 298.15  # K
pressure = 101325  # Pa

print(
    "Molar volume of the mixture: ",
    np.round(gas_mixture.volume(moles, 273, pressure), 1),
    "m³/mol",
)
print(
    "Heat capacity: ",
    np.round(gas_mixture.mix_heat_capacity(moles, temperature, 101325), 1),
    " J/K",
)

standard_enthalpies = gas_mixture._formation_enthalpies_set()
print("Formation enthalpies: ", standard_enthalpies, " J/(mol.K)")

enthalpy_correction = gas_mixture.formation_enthalpies_correction(400, 101325)
print("Enthalpy correction: ", np.round(enthalpy_correction, 1), " J/(mol.K)")
print(
    "Corrected enthalpies: ",
    np.round(standard_enthalpies + enthalpy_correction, 1),
    " J/(mol.K)",
)

Molar volume of the mixture:  0.0 m³/mol
Heat capacity:  38.4  J/K
Formation enthalpies:  [-296800.       0. -395700. -393474.]  J/(mol.K)
Enthalpy correction:  [4273.3 3025.5 5561.8 4004.7]  J/(mol.K)
Corrected enthalpies:  [-292526.7    3025.5 -390138.2 -389469.3]  J/(mol.K)


A method to calculate the partial pressures is defined at the level of the abstract class AbstractMix

In [14]:
moles = [8, 20, 15, 12]
temperature = 700  # K
pressure = 500000  # Pa

partial_pressures = gas_mixture.partial_pressures(moles, temperature, pressure)
print(np.round(partial_pressures, 0), "Pa")

[ 72727. 181818. 136364. 109091.] Pa
